In [3]:
from keras.datasets import mnist

Using TensorFlow backend.


In [4]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [5]:
n = 28*28    #feature space dimension 
x_train = x_train.reshape((60000, n))

In [6]:
N_train = len(x_train)
N_test = len(x_test)

x_test = x_test.reshape((N_test, n))

In [7]:
x_train_scaled = x_train/225
x_test_scaled = x_test/225

In [8]:
num_inputs = 784
neurons_hid1 = 512
neurons_hid2 = 512
num_outputs = 10

In [9]:
learning_rate = 0.01

In [10]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

In [11]:
actf1 = tf.nn.relu

In [12]:
actf2 = tf.nn.softmax

In [13]:
X = tf.placeholder(tf.float32, shape=[None, num_inputs])

In [14]:
initializer = tf.variance_scaling_initializer()

In [15]:
w1 = tf.Variable(initializer([num_inputs, neurons_hid1]), dtype=tf.float32)
w2 = tf.Variable(initializer([neurons_hid1, neurons_hid2]), dtype=tf.float32)
w3 = tf.Variable(initializer([neurons_hid2, num_outputs]), dtype=tf.float32)

In [16]:
b1 = tf.Variable(tf.zeros(neurons_hid1))
b2 = tf.Variable(tf.zeros(neurons_hid2))
b3 = tf.Variable(tf.zeros(num_outputs))

In [25]:
hid_layer1 = actf1(tf.matmul(X,w1)+b1)
hid_layer2 = actf1(tf.matmul(hid_layer1,w2)+b2)
output_layer = tf.matmul(hid_layer2,w3)+b3

In [26]:
Y = tf.placeholder(tf.uint8, shape=[None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=Y))

In [27]:
optimizer = tf.train.AdamOptimizer(learning_rate)

In [28]:
train = optimizer.minimize(loss)

In [29]:
init = tf.global_variables_initializer()

In [30]:
import pandas as pd
y_train_onehot = pd.get_dummies(y_train.astype(str))
y_test_onehot = pd.get_dummies(y_test.astype(str))
y_train_onehot = y_train_onehot.values
y_test_onehot = y_test_onehot.values

In [31]:
import numpy as np
num_epochs = 30
batch_size = 32

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epochs):
        
        num_batches = len(x_train_scaled)//batch_size
        x_batches = np.array_split(x_train_scaled, num_batches)
        y_batches = np.array_split(y_train_onehot, num_batches)
        
        for i in range(num_batches):
            
            X_batch = x_batches[i]
            Y_batch = y_batches[i]
            
            sess.run(train,feed_dict={X:X_batch, Y:Y_batch})
            
        training_loss = loss.eval(feed_dict={X:X_batch, Y:Y_batch})
        
        print('epoch: {} Loss: {}'.format(epoch,training_loss))
        
    # Evaluating
    correct_prediction = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y,1))
    
    # [True, False, True, ...] --> [1, 0, 1, ...]
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(acc, feed_dict={X: x_test_scaled, Y: y_test_onehot}))

epoch: 0 Loss: 0.013852857053279877
epoch: 1 Loss: 0.08621084690093994
epoch: 2 Loss: 0.005981272552162409
epoch: 3 Loss: 0.002842166693881154
epoch: 4 Loss: 0.014916513115167618
epoch: 5 Loss: 0.027732301503419876
epoch: 6 Loss: 0.002706584520637989
epoch: 7 Loss: 0.42140066623687744
epoch: 8 Loss: 0.005839128512889147
epoch: 9 Loss: 0.008478319272398949
epoch: 10 Loss: 0.0021829463075846434
epoch: 11 Loss: 0.004434795118868351
epoch: 12 Loss: 0.23993350565433502
epoch: 13 Loss: 0.10614810138940811
epoch: 14 Loss: 0.0725460797548294
epoch: 15 Loss: 0.00011744648509193212
epoch: 16 Loss: 0.0005014874041080475
epoch: 17 Loss: 0.0005604631151072681
epoch: 18 Loss: 0.00015316830831579864
epoch: 19 Loss: 0.001013266621157527
epoch: 20 Loss: 0.00031889573438093066
epoch: 21 Loss: 0.0014722872292622924
epoch: 22 Loss: 5.250057438388467e-05
epoch: 23 Loss: 7.4505797087454084e-09
epoch: 24 Loss: 3.5575042147684144e-06
epoch: 25 Loss: 2.831207837061811e-07
epoch: 26 Loss: 0.06725523620843887
ep